In [1]:
from google.colab import files

uploaded = files.upload()


Saving coffee_reviews_61pages.csv to coffee_reviews_61pages.csv


In [38]:
import pandas as pd

df = pd.read_csv("coffee_reviews_61pages.csv")
print(df.head())


                          Coffee Name         Roaster Location  \
0  Reserve Panama Volcán Valley Gesha         Phoenix, Arizona   
1             El Salvador Monte Verde  Plymouth, Massachusetts   
2             Guatemala Bella Carmona       Madison, Wisconsin   
3               African Waltz Natural          Chia-Yi, Taiwan   
4            American Rhapsody Washed          Chia-Yi, Taiwan   

                                       Coffee Origin   Roast Level  \
0                              Volcán Valley, Panama         Light   
1  Monte Verde, Santa Ana Department, central El ...         Light   
2  Antigua valley, Sacatepequez Department, Guate...  Medium-Light   
3                                    Ethiopia; Kenya  Medium-Light   
4                        Ethiopia; Kenya; Costa Rica  Medium-Light   

          Est. Price  Review Date  Aroma  Acidity/Structure  Body  Flavor  \
0    $56.00/6 ounces  August 2025    9.0                9.0   9.0      10   
1   $18.00/12 ounces  August

In [39]:
print("Before removing duplicates:", len(df))

# Remove duplicates
df = df.drop_duplicates()

print("After removing duplicates:", len(df))

df.head()


Before removing duplicates: 1220
After removing duplicates: 1212


,Coffee Name,Roaster Location,Coffee Origin,Roast Level,Est. Price,Review Date,Aroma,Acidity/Structure,Body,Flavor,Aftertaste,Rating
0,Reserve Panama Volcán Valley Gesha,"Phoenix, Arizona","Volcán Valley, Panama",Light,$56.00/6 ounces,August 2025,9.0,9.0,9.0,10,9.0,96
1,El Salvador Monte Verde,"Plymouth, Massachusetts","Monte Verde, Santa Ana Department, central El ...",Light,$18.00/12 ounces,August 2025,9.0,9.0,8.0,9,8.0,93
2,Guatemala Bella Carmona,"Madison, Wisconsin","Antigua valley, Sacatepequez Department, Guate...",Medium-Light,$25.00/12 ounces,August 2025,9.0,8.0,9.0,9,8.0,93
3,African Waltz Natural,"Chia-Yi, Taiwan",Ethiopia; Kenya,Medium-Light,NT $700/227 grams,August 2025,9.0,9.0,9.0,9,9.0,95
4,American Rhapsody Washed,"Chia-Yi, Taiwan",Ethiopia; Kenya; Costa Rica,Medium-Light,NT $700/227 grams,August 2025,9.0,9.0,9.0,9,8.0,94


In [40]:
df.isnull().sum()

,0
Coffee Name,0
Roaster Location,1
Coffee Origin,2
Roast Level,13
Est. Price,0
Review Date,0
Aroma,16
Acidity/Structure,146
Body,1
Flavor,0


In [41]:
# Fill categorical columns with 'Unknown'
df['Roaster Location'] = df['Roaster Location'].fillna('Unknown')
df['Coffee Origin'] = df['Coffee Origin'].fillna('Unknown')
df['Roast Level'] = df['Roast Level'].fillna('Unknown')

# Fill numeric columns with their mean
df['Aroma'] = df['Aroma'].fillna(df['Aroma'].mean())
df['Acidity/Structure'] = df['Acidity/Structure'].fillna(df['Acidity/Structure'].mean())
df['Body'] = df['Body'].fillna(df['Body'].mean())
df['Aftertaste'] = df['Aftertaste'].fillna(df['Aftertaste'].mean())

df.isnull().sum()


,0
Coffee Name,0
Roaster Location,0
Coffee Origin,0
Roast Level,0
Est. Price,0
Review Date,0
Aroma,0
Acidity/Structure,0
Body,0
Flavor,0


In [42]:
# Standardize column names: snake_case
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('/', '_')
)

df.columns


Index(['coffee_name', 'roaster_location', 'coffee_origin', 'roast_level',
       'est._price', 'review_date', 'aroma', 'acidity_structure', 'body',
       'flavor', 'aftertaste', 'rating'],
      dtype='object')

In [43]:
df = df.rename(columns={'est._price': 'est_price'})
#Exchange rates
exchange_rates = {
    "$": 1.0,
    "NT$": 0.032,
    "RMB": 0.14,
    "£": 1.25,
    "RM": 0.23,
    "HK$": 0.13
}

#Price conversion function
def price_usd_per_100g(price_str):
    if pd.isna(price_str):
        return None

    price_str = price_str.replace(" ", "")

    # Match currency, amount, quantity, unit
    match = re.search(r"(\$|NT\$|RMB|£|RM|HK\$)?(\d+\.?\d*)/(\d+\.?\d*)(oz|ounces|g|grams|kilogram|kg|capsules)?", price_str, re.IGNORECASE)
    if match:
        currency, amount, qty, unit = match.groups()
        currency = currency or "$"
        amount = float(amount)
        qty = float(qty)
        unit = (unit or "g").lower()

        # Convert weight to grams
        if unit in ["oz", "ounce", "ounces"]:
            grams = qty * 28.35
        elif unit in ["kilogram", "kg"]:
            grams = qty * 1000
        elif unit in ["capsules", "capsule"]:
            return round(amount, 2)
        else:
            grams = qty

        # Convert to USD
        rate = exchange_rates.get(currency, 1.0)
        usd_amount = amount * rate

        return round((usd_amount / grams) * 100, 2)

    return None

df['price_usd_per_100g'] = df['est_price'].apply(price_usd_per_100g)

df[['est_price', 'price_usd_per_100g']].head(10)




,est_price,price_usd_per_100g
0,$56.00/6 ounces,32.92
1,$18.00/12 ounces,5.29
2,$25.00/12 ounces,7.35
3,NT $700/227 grams,9.87
4,NT $700/227 grams,9.87
5,$23.00/12 ounces,6.76
6,$20.00/10.9 ounces,6.47
7,$23.00/12 ounces,6.76
8,$32.00/12 ounces,9.41
9,$32.00/12 ounces,9.41


In [44]:
#Convert to datetime
df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')

#Extract year and month
df['review_year'] = df['review_date'].dt.year
df['review_month'] = df['review_date'].dt.month

df[['review_date', 'review_year', 'review_month']].head(50)


/tmp/ipython-input-3970447156.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')


,review_date,review_year,review_month
0,2025-08-01,2025,8
1,2025-08-01,2025,8
2,2025-08-01,2025,8
3,2025-08-01,2025,8
4,2025-08-01,2025,8
5,2025-08-01,2025,8
6,2025-08-01,2025,8
7,2025-08-01,2025,8
8,2025-08-01,2025,8
9,2025-08-01,2025,8


In [45]:
# Split roaster_location by commas into multiple columns
location_split = df['roaster_location'].str.split(',', expand=True)

location_split.head()


,0,1,2
0,Phoenix,Arizona,None
1,Plymouth,Massachusetts,None
2,Madison,Wisconsin,None
3,Chia-Yi,Taiwan,None
4,Chia-Yi,Taiwan,None


In [46]:
# Assign city, state, country based on max 3 parts
df['roaster_city'] = location_split[0].str.strip()
df['roaster_state'] = location_split[1].str.strip() if location_split.shape[1] > 1 else None
df['roaster_country'] = location_split[2].str.strip() if location_split.shape[1] > 2 else None


In [47]:
known_countries = ['taiwan', 'china', 'italy', 'malaysia', 'guatemala',
                   'japan', 'thailand', 'south korea', 'canada', 'england',
                   'uk', 'usa', 'hawaii', 'mexico']

def fix_country(row):
    city = str(row['roaster_city']).lower()
    state = str(row['roaster_state']).lower() if row['roaster_state'] else ''
    country = str(row['roaster_country']).lower() if row['roaster_country'] else ''

    for c in known_countries:
        if c in [city, state, country]:
            if c in ['usa', 'hawaii', 'hawai’i']:
                return 'USA'
            elif c in ['uk', 'england']:
                return 'UK'
            else:
                return c.title()

    return 'Unknown'

df['roaster_country'] = df.apply(fix_country, axis=1)
df[['roaster_location','roaster_city','roaster_state','roaster_country']].head(80)


,roaster_location,roaster_city,roaster_state,roaster_country
0,"Phoenix, Arizona",Phoenix,Arizona,Unknown
1,"Plymouth, Massachusetts",Plymouth,Massachusetts,Unknown
2,"Madison, Wisconsin",Madison,Wisconsin,Unknown
3,"Chia-Yi, Taiwan",Chia-Yi,Taiwan,Taiwan
4,"Chia-Yi, Taiwan",Chia-Yi,Taiwan,Taiwan
...,...,...,...,...
77,"Floyd, Virginia",Floyd,Virginia,Unknown
78,"Madison, Wisconsin",Madison,Wisconsin,Unknown
79,"Madison, Wisconsin",Madison,Wisconsin,Unknown
80,"Madison, Wisconsin",Madison,Wisconsin,Unknown


In [48]:
# Fix unknown countries
df['roaster_country'] = df['roaster_country'].replace('Unknown', 'USA')
df[['roaster_location','roaster_city','roaster_state','roaster_country']].head(80)

,roaster_location,roaster_city,roaster_state,roaster_country
0,"Phoenix, Arizona",Phoenix,Arizona,USA
1,"Plymouth, Massachusetts",Plymouth,Massachusetts,USA
2,"Madison, Wisconsin",Madison,Wisconsin,USA
3,"Chia-Yi, Taiwan",Chia-Yi,Taiwan,Taiwan
4,"Chia-Yi, Taiwan",Chia-Yi,Taiwan,Taiwan
...,...,...,...,...
77,"Floyd, Virginia",Floyd,Virginia,USA
78,"Madison, Wisconsin",Madison,Wisconsin,USA
79,"Madison, Wisconsin",Madison,Wisconsin,USA
80,"Madison, Wisconsin",Madison,Wisconsin,USA


In [49]:
roast_mapping = {
    'Light': 1,
    'Medium-Light': 2,
    'Medium': 3,
    'Medium-Dark': 4,
    'Dark': 5,
    'Very Dark': 6,
    'Unknown': 0
}

df['roast_level_normalized'] = df['roast_level'].map(roast_mapping)

df['roast_level_normalized'] = df['roast_level_normalized'].fillna(0).astype(int)

df[['roast_level', 'roast_level_normalized']].head(10)



,roast_level,roast_level_normalized
0,Light,1
1,Light,1
2,Medium-Light,2
3,Medium-Light,2
4,Medium-Light,2
5,Light,1
6,Light,1
7,Medium-Light,2
8,Light,1
9,Light,1


In [50]:
def classify_origin(origin):
    if pd.isna(origin):
        return 'Unknown'
    elif ',' in origin or '/' in origin or '&' in origin:
        return 'Blend'
    else:
        return 'Single Origin'

df['origin_type'] = df['coffee_origin'].apply(classify_origin)

df[['coffee_origin', 'origin_type']].head(10)


,coffee_origin,origin_type
0,"Volcán Valley, Panama",Blend
1,"Monte Verde, Santa Ana Department, central El ...",Blend
2,"Antigua valley, Sacatepequez Department, Guate...",Blend
3,Ethiopia; Kenya,Single Origin
4,Ethiopia; Kenya; Costa Rica,Single Origin
5,"Dhibabate, Hambela Wamena, Guji Zone, Ethiopia",Blend
6,"Kokose, Bensa, Sidamo growing region, southern...",Blend
7,"Aceh Province, Sumatra, Indonesia",Blend
8,"Armenia, Quindio Department, Colombia",Blend
9,"Armenia, Quindio Department, Colombia",Blend


In [51]:
# Convert sensory columns to numeric
sensory_cols = ['aroma', 'acidity_structure', 'body', 'flavor', 'aftertaste']

for col in sensory_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [52]:
df['flavor_profile'] = df['aroma'].astype(str) + '-' + \
                       df['acidity_structure'].astype(str) + '-' + \
                       df['body'].astype(str) + '-' + \
                       df['flavor'].astype(str) + '-' + \
                       df['aftertaste'].astype(str)


In [53]:
# Create a boolean column
df['is_single_origin'] = df['coffee_origin'].apply(lambda x: True if x == 'Single Origin' else False)


In [54]:
# List of sensory columns
sensory_cols = ['aroma', 'acidity_structure', 'body', 'flavor', 'aftertaste']

# Convert to numeric
df[sensory_cols] = df[sensory_cols].apply(pd.to_numeric, errors='coerce')

# Calculate average
df['avg_sensory_score'] = df[sensory_cols].mean(axis=1)


In [55]:
# Encode 'origin_type': Single Origin = 1, Blend = 0
df['origin_encoded'] = df['origin_type'].map({'Single Origin': 1, 'Blend': 0, 'Unknown': 0})

# Select columns for correlation
correlation_cols = [
    'price_usd_per_100g',
    'avg_sensory_score',
    'roast_level_normalized',
    'origin_encoded'
]

correlation_matrix = df[correlation_cols].corr()

print(correlation_matrix)


                        price_usd_per_100g  avg_sensory_score  \
price_usd_per_100g                1.000000           0.140980   
avg_sensory_score                 0.140980           1.000000   
roast_level_normalized           -0.054494          -0.441501   
origin_encoded                   -0.041187          -0.372441   

                        roast_level_normalized  origin_encoded  
price_usd_per_100g                   -0.054494       -0.041187  
avg_sensory_score                    -0.441501       -0.372441  
roast_level_normalized                1.000000        0.345925  
origin_encoded                        0.345925        1.000000  


In [56]:
df.to_csv("coffee_reviews_cleaned.csv", index=False)

from google.colab import files
files.download("coffee_reviews_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>